In [ ]:
!wget https://raw.githubusercontent.com/HassamSheikh/rnn-for-eminem/master/lyrics.txt

--2024-01-25 10:24:34--  https://raw.githubusercontent.com/HassamSheikh/rnn-for-eminem/master/lyrics.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 522713 (510K) [text/plain]
Saving to: ‘lyrics.txt’

lyrics.txt          100%[===================>] 510.46K  --.-KB/s    in 0.03s   

2024-01-25 10:24:34 (18.6 MB/s) - ‘lyrics.txt’ saved [522713/522713]



In [ ]:
import tqdm
import torch
import torch.nn
import numpy as np
from torch.optim import Adam
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"This notebook uses {device}!")

This notebook uses cuda!


In [ ]:
with open('lyrics.txt') as f:
  text = f.read()

print(f"Number of characters are {len(text)}")

Number of characters are 522527


In [ ]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/762 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/353M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [ ]:
tokenized_text = tokenizer(text, return_tensors='pt')
def get_random_batch(batch_size=32, block_size=128):
  indices = np.random.randint(low=0,high=tokenized_text['input_ids'].shape[1]-block_size,size=(batch_size,))
  input_ids = torch.stack([tokenized_text['input_ids'][0,idx:idx+block_size] for idx in indices])
  attention_mask = torch.stack([tokenized_text['attention_mask'][0,idx:idx+block_size] for idx in indices])
  return input_ids, attention_mask

Token indices sequence length is longer than the specified maximum sequence length for this model (146765 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
optimizer = Adam(model.parameters(),lr=1e-3)
epochs = 100

model.train()
for epoch in tqdm.tqdm(range(epochs)):
  optimizer.zero_grad()
  input_ids , attention_mask = get_random_batch()
  logits = model(input_ids=input_ids.to(device),
                 attention_mask=attention_mask.to(device),
                 labels = input_ids.to(device))
  loss = logits.loss
  loss.backward()
  optimizer.step()

100%|██████████| 100/100 [01:17<00:00,  1.28it/s]


In [ ]:
test = "I would never fall in love"
tokens = tokenizer(test, return_tensors='pt')
print(tokenizer.decode(model.generate(input_ids=tokens['input_ids'].to(device),
                     attention_mask=tokens['attention_mask'].to(device),
                     max_new_tokens=1000,
                    do_sample=True,
                    temperature=1.2,
                    pad_token_id=tokenizer.eos_token_id)[0]))

I would never fall in love with him so
'Cause if it was then I wouldn't die
If I was still alive, I'd know

[Chorus x2]

Nowhere to rest at the end, I said I'm leaving rap about it at all
It's up to you to sing a sweet letter that says
'Shady the World Peace Out'
And don't compare it to what I receive, you will.
Take it from you right now, I ain't beautiful enough.

I'm 28...
You got what I would do without you.
You're probably sick of me motherfucking
Yeah... yeah you're your motherfucking good..

[8X2] You are. I'm going insane
Fucked by me
You're on the kitchen floor til then you
I think you don't look so smart

But you think I wouldn't look so too fuckin better
I look dumb to yourself
That would make no worse
Cause if I tried at how to see me
Sole your wife just might relapse
So I'd give away in a second
I could make it
I'm the world right now?


If I'm pregnant when I sit on the table I'm on the floor screaming and swallow the
When I say I look your puppy
You talk to you

Would yo